# Introduction to RNN-LSTM

## Content
### 1.
### 2.

### *Importing packages:*

In [ ]:
#RNN
import tensorflow as tf
#Graphics
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
#Setting figure size
mpl.rcParams['figure.figsize'] = (8,6)
mpl.rcParams['axes.grid'] = False

### *Importing data:*

In [ ]:
#installing yfinance (Colab)
!pip install yfinance
#pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 5.5MB 15.9MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=40b366037103184ff0f82fa46456aa80b64ff02ddcb1d7dd5f60df764459ee39
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
###THINGS: minimum interval: 1 minute
###NOTE: testing a lot of methods
import yfinance as yf
btc = yf.Ticker("BTC-USD")
btc.info

{'algorithm': None,
 'ask': None,
 'askSize': None,
 'averageDailyVolume10Day': 58119660885,
 'averageVolume': 65092437925,
 'averageVolume10days': 58119660885,
 'beta': None,
 'bid': None,
 'bidSize': None,
 'circulatingSupply': 18695600,
 'companyOfficers': [],
 'currency': 'USD',
 'dayHigh': 58031.312,
 'dayLow': 57728.312,
 'description': 'Bitcoin (BTC) is a cryptocurrency . Users are able to generate BTC through the process of mining. Bitcoin has a current supply of 18,695,162. The last known price of Bitcoin is 54,647.1255491 USD and is up 0.41 over the last 24 hours. It is currently trading on 9539 active market(s) with $46,928,605,556.25 traded over the last 24 hours. More information can be found at https://bitcoin.org/.',
 'dividendRate': None,
 'dividendYield': None,
 'exDividendDate': None,
 'exchange': 'CCC',
 'exchangeTimezoneName': 'Europe/London',
 'exchangeTimezoneShortName': 'BST',
 'expireDate': None,
 'fiftyDayAverage': 57083.09,
 'fiftyTwoWeekHigh': 64863.098,
 'fi

In [ ]:
###THINGS: minimum interval: 1 minute
###NOTE: testing a lot of methods

import yfinance as yf

from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

#Uni
data = pdr.get_data_yahoo("BTC-USD")

#Multiple
#starDate="2017-01-01" #year-month-day
#endDate="2017-04-30"
#data = pdr.get_data_yahoo("BTC-USD ETH-USD", start=starDate, end=endDate)
print(data)
#print(data["Adj Close"])
#print(type(data["Adj Close"]["BTC-USD"]))
#print(data["Adj Close"]["BTC-USD"])

#Multiple Ticker objects
tickers = yf.Tickers('msft aapl goog')
# ^ returns a named tuple of Ticker objects
#tickers.tickers.info
#print(tickers.tickers.MSFT)
# access each ticker using (example)
#tickers.tickers["MSFT"].info
#tickers.tickers["AAPL"].history(period="1mo")
#tickers.tickers.GOOG.actions

btc_usd = yf.Ticker("BTC-USD")
#type(btc_usd)
#Exploring some methods
#btc_usd.info
#btc_usd.actions
#btc_usd.financials
#btc_usd.history(period="max")
#btc_usd.calendar
#### valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
#btc_usd.history(period="2d",interval="90m") 



[*********************100%***********************]  1 of 1 completed
                    Open          High  ...     Adj Close       Volume
Date                                    ...                           
2014-09-17    465.864014    468.174011  ...    457.334015     21056800
2014-09-18    456.859985    456.859985  ...    424.440002     34483200
2014-09-19    424.102997    427.834991  ...    394.795990     37919700
2014-09-20    394.673004    423.295990  ...    408.903992     36863600
2014-09-21    408.084991    412.425995  ...    398.821014     26580100
...                  ...           ...  ...           ...          ...
2021-04-26  49077.792969  54288.003906  ...  54021.753906  58284039825
2021-04-27  54030.304688  55416.964844  ...  55033.117188  49448222757
2021-04-28  55036.636719  56227.207031  ...  54824.703125  48000572955
2021-04-29  54858.089844  55115.843750  ...  53555.109375  46088929780
2021-05-01  57728.312500  58031.312500  ...  57975.230469  52958375936

[2414 r

## Bibliography
1. [Introducción a Keras LSTM para series de tiempo multivariadas. Montenegro et.al.](https://nbviewer.jupyter.org/github/AprendizajeProfundo/Diplomado/blob/master/Temas/M%C3%B3dulo%205-%20Redes%20recurrentes/Cuadernos/rnrTimes_series_Multivariate.ipynb).
2. [Yahoo Finance API – A Complete Guide](https://algotrading101.com/learn/yahoo-finance-api-guide/)
3. [yfinance Library – A Complete Guide](https://algotrading101.com/learn/yfinance-guide/)
4. [yfinance documentation](https://pypi.org/project/yfinance/)

TO CHECK:

[Introducción a Keras LSTM para series de tiempo multivariadas. Montenegro et.al.](). 
[ninja3697/Stocks-Price-Prediction-using-Multivariate-Analysis11214
](https://github.com/ninja3697/Stocks-Price-Prediction-using-Multivariate-Analysis). 
[Building Multivariate Time Series Models for Stock Market Prediction with Python](https://www.relataly.com/stock-market-prediction-with-multivariate-time-series-in-python/1815/). 
[(Weather) How to Convert a Time Series to a Supervised Learning Problem in Python ](https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/).
[(weather) Multivariate Time Series Forecasting with LSTMs in Keras
](https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/).
[Book to download](François Chollet para su libro Deep Learning with Python).
[]().

[Stock Price Prediction Using Convolutional Neural Networks on a Multivariate Timeseries. Article](https://www.researchgate.net/publication/338477393_Stock_Price_Prediction_Using_Convolutional_Neural_Networks_on_a_Multivariate_Timeseries). 

